<a href="https://colab.research.google.com/github/Shakilkhan24/Playground_DL/blob/main/Corrective_RAG_warmup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
api_key=' '

In [ ]:
!pip install langchain
!pip install langchain-openai

In [ ]:
!pip install unstructured

In [ ]:
!pip install chromadb

# web page scrapping and storing into vector database


```
# splitting , embedding
```



In [ ]:
import os
from bs4 import BeautifulSoup
import os
import requests
import chromadb
from typing import List, Dict
from langchain_community.document_loaders import UnstructuredHTMLLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.chat_models import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAI
from langchain import PromptTemplate

url='https://huggingface.co/learn/nlp-course/chapter3/4'
html = requests.get(url).text
with open('explore.html', 'w') as f:
    f.write(html)

chat = ChatOpenAI(temperature=0, openai_api_key=api_key)
embedding = OpenAIEmbeddings(openai_api_key=api_key)

loader = UnstructuredHTMLLoader("explore.html")
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
 )

text_chunks = splitter.split_documents(docs)

db = Chroma.from_documents(text_chunks, embedding)
chat_model = ChatOpenAI(openai_api_key=api_key)

# we can specify multiple chuncks


```
# usually we take the first one
```



In [ ]:
print(text_chunks[3].page_content)

# first checking **retrieved** data is query relevant or not.



```
# directing llm to return YES or NO as string
```



In [ ]:
prompttemplate1 = """
Is this retrieved data : [{retrieved_data1}], is relevant accoroding to the query: [{query1}]?
Return a simple boolean variable 'yes' or 'no'.
"""
# returning a boolean variable yes or no .

x='tell me about the soil of mars planet?'
retrieved_data1=db.similarity_search(x)[0]

prompt1=PromptTemplate.from_template(prompttemplate1)
prompt1=prompt1.format(retrieved_data1=retrieved_data1,query1=x)


In [ ]:
yes_or_no=chat_model.predict(prompt1)


In [ ]:
yes_or_no

'No'

In [ ]:
type(yes_or_no)

str

In [ ]:
print(retrieved_data1.page_content)

In [ ]:
!pip install -U langchain-community tavily-python

In [ ]:
# TAVILY_API_KEY='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'
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass()

··········


# **Tavily** is for web search

In [ ]:
from langchain_community.tools.tavily_search import TavilySearchResults

tool = TavilySearchResults()

In [ ]:
prompt_template2="""
answer the question based on the data: [{retrieved_data}, accoroding to the prompt: [{prompt}]]
"""

# If **YES** , then predicting based on **Retrieved_data**

# If **NO**, then predicting based on **web_search** data


```
# **x** is the main question we started with
```



In [ ]:
# if yes , then predicting based on the retrieved data , additionally model's pretrained knowledge
# if no, then go for web searching


if yes_or_no=='yes':
  result=chat_model.invoke(prompt_template2.format(retrieved_data=retrieved_data1,prompt=x))
else:
  search_result=tool.invoke({'query':x})
  print(search_result)
  result=chat_model.invoke(prompt_template2.format(retrieved_data=search_result,prompt=x))


[{'url': 'https://scitechdaily.com/geologists-simulate-martian-soil-conditions-to-figure-out-how-to-grow-plants-on-mars/', 'content': 'A difficult task Using simulated Martian soils, Fackrell and fellow researchers have found the textures of artificial simulants to be crusty and dried which may reflect some unexpected conditions of Mars soils that make them more difficult to use. These challenges add up to a very difficult, though not impossible task.'}, {'url': 'https://geoscience.blog/exploring-the-science-of-martian-soil-unveiling-the-secrets-to-replicating-the-red-planets-unique-composition-on-earth/', 'content': '1. Select basaltic material The first step is to obtain basaltic material that closely resembles the composition of the Martian regolith. Basalt rocks similar to those found on Mars can be sourced from volcanic regions on Earth.'}, {'url': 'https://academic.oup.com/astrogeo/article/57/2/2.18/2468646', 'content': 'Watney has water and it is safe to assume that any humans o

# we need to ***parsing*** our results

In [ ]:
search_result

In [ ]:
result

AIMessage(content='Sorry, I cannot provide information about the soil of Mars planet based on the given data about fine-tuning models and training loops.')

In [ ]:
openai_api_key=''

# Entire code
# make ensure openai_api and tavily_api is set


```
# just give any webpage and question , it would generate solution for you
```



In [ ]:
import os
import requests
from bs4 import BeautifulSoup
from typing import List, Dict
from langchain_community.document_loaders import UnstructuredHTMLLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OpenAIEmbeddings
from langchain_community.chat_models import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAI
from langchain import PromptTemplate

def process_data(url: str, x: str) -> str:
    html = requests.get(url).text
    with open('explore.html', 'w') as f:
        f.write(html)

    chat = ChatOpenAI(temperature=0, openai_api_key=openai_api_key)
    embedding = OpenAIEmbeddings(openai_api_key=openai_api_key)

    loader = UnstructuredHTMLLoader("explore.html")
    docs = loader.load()

    splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
    )

    text_chunks = splitter.split_documents(docs)

    db = Chroma.from_documents(text_chunks, embedding)
    chat_model = ChatOpenAI(openai_api_key=openai_api_key)

    prompttemplate1 = """
    Is this retrieved data: [{retrieved_data1}], relevant according to the query: [{query1}]?
    Return a simple boolean variable 'yes' or 'no'.
    """

    # returning a boolean variable yes or no.
    retrieved_data1 = db.similarity_search(x)[0]
    prompt1 = PromptTemplate.from_template(prompttemplate1)
    prompt1 = prompt1.format(retrieved_data1=retrieved_data1, query1=x)

    from langchain_community.tools.tavily_search import TavilySearchResults

    tool = TavilySearchResults()
    prompt_template2 = """
    Answer the question based on the data: [{retrieved_data}], according to the prompt: [{prompt}],
    also you can add more information from your knowledge (pretrained) , as if answer tends to be perfect or accurate or organized etc
    """

    # If yes, then predicting based on the retrieved data, additionally model's pretrained knowledge
    # If no, then go for web searching
   # Placeholder value, replace with actual logic

    if yes_or_no.lower() == 'yes':
        result = chat_model.invoke(prompt_template2.format(retrieved_data=retrieved_data1, prompt=x))
    else:
        search_result = tool.invoke({'query': x})
        print(search_result)
        result = chat_model.invoke(prompt_template2.format(retrieved_data=search_result, prompt=x))

    return result

In [ ]:
url='https://huggingface.co/learn/nlp-course/chapter3/4'
question='what is the elements of mars soil?'
print(process_data(url,question))

[{'url': 'https://www.spacechatter.com/2024/01/29/solvents-building-materials-mars/', 'content': 'January 29, 2024 by Space Chatter Wire Will humans be able to build on the Moon and Mars? Thanks to Washington State University researchers, the sci-fi fantasy is moving closer to reality. Scientists discovered powerful liquid solvents capable of extracting vital materials from lunar and Martian rock dust for 3D printing.'}, {'url': 'https://www.newscientist.com/article/dn22372-black-glass-holds-first-mars-soil-sample-on-earth/', 'content': 'The team says weakly acidic water may have leached rare-earth elements from Martian soil and deposited them in fractures in surface rocks. Heat from the asteroid impact that ejected Tissint melted ...'}, {'url': 'https://www.nature.com/articles/nature03637', 'content': 'Author information\nAuthors and Affiliations\nJet Propulsion Laboratory, California Institute of Technology, California, 91109, Pasadena, USA\nAlbert S. Yen,Â\xa0Joy A. CrispÂ\xa0&Â\xa0

In [ ]:
# after building a sequential pipeline, i would encapsulate everything in a class , also different exception handler or other oop and
# basic concepts





from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_community.loader import UnstructuredHTMLLoader
from langchain_community.splitter import RecursiveCharacterTextSplitter
from langchain_community.chroma import Chroma
from langchain_community.prompt_template import PromptTemplate
from langchain_community.chat_openai import ChatOpenAI
from langchain_community.openai_embeddings import OpenAIEmbeddings


import requests
from bs4 import BeautifulSoup
from urllib.parse import urlparse, urljoin


# extracting links from an entire website

def bfs_crawl(url, limit):
    visited = set()
    queue = [url]
    count = 0
    extracted_links = []

    while queue and count < limit:
        current_url = queue.pop(0)
        visited.add(current_url)

        try:
            response = requests.get(current_url)
        except requests.exceptions.RequestException:
            continue

        soup = BeautifulSoup(response.text, 'html.parser')

        for link in soup.find_all('a'):
            href = link.get('href')
            if href:
                absolute_url = urljoin(current_url, href)
                parsed_url = urlparse(absolute_url)

                if parsed_url.netloc == urlparse(url).netloc:
                    if absolute_url not in visited and absolute_url not in queue:
                        queue.append(absolute_url)
                        visited.add(absolute_url)
                        count += 1
                        extracted_links.append(absolute_url)

        print(f"Crawled: {current_url}")

    return extracted_links

def str_func(s:str):    # function for preprocessing web_retrieved_text (just like cleaning data for better output)
  pass

# extracting informations from each link
def func(links):
  docs=''
  for link in links:
    loader=WebBaseLoader(link)
    d=loader.load()
    docs+=str_func(d[0].page_content)
  return docs


# converting into a html file
def html_converter(docs:str):
  with open('entire_text.txt','w') as f:
    f.write(docs)


# now finally adding the html file datas to a vectorstore database

# if it returns , db.sqlite3 , then download that , and connect the data to the app as a tool and use agent executor

# just like you did in this note book

# https://colab.research.google.com/drive/1DfcFNBZxI5pWkZXgZp2bJIJtDdUmUnah




# ADD TOOLs AND AGENTs TO THIS FUNCTION
def process_html_file(html_file_path, openai_api_key, x):
    with open(html_file_path, 'r') as f:
        html = f.read()

    chat = ChatOpenAI(temperature=0, openai_api_key=openai_api_key)
    embedding = OpenAIEmbeddings(openai_api_key=openai_api_key)

    loader = UnstructuredHTMLLoader(html_file_path)
    docs = loader.load()

    splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
    )

    text_chunks = splitter.split_documents(docs)

    db = Chroma.from_documents(text_chunks, embedding)
    chat_model = ChatOpenAI(openai_api_key=openai_api_key)

    prompttemplate1 = """
    Is this retrieved data: [{retrieved_data1}], relevant according to the query: [{query1}]?
    Return a simple boolean variable 'yes' or 'no'.
    """

    # returning a boolean variable yes or no.
    retrieved_data1 = db.similarity_search(x)[0]
    prompt1 = PromptTemplate.from_template(prompttemplate1)
    prompt1 = prompt1.format(retrieved_data1=retrieved_data1, query1=x)

    tool = TavilySearchResults()
    prompt_template2 = """
    Answer the question based on the data: [{retrieved_data}], according to the prompt: [{prompt}],
    also you can add more information from your knowledge (pretrained) , as if answer tends to be perfect or accurate or organized etc
    """

    # If yes, then predicting based on the retrieved data, additionally model's pretrained knowledge
    # If no, then go for web searching
    # Placeholder value, replace with actual logic

    if yes_or_no.lower() == 'yes':
        result = chat_model.invoke(prompt_template2.format(retrieved_data=retrieved_data1, prompt=x))
    else:
        search_result = tool.invoke({'query': x})
        print(search_result)
        result = chat_model.invoke(prompt_template2.format(retrieved_data=search_result, prompt=x))

    return result

# for code snippet generation , extracting the code parts
# output parsing technicque little bit

In [ ]:
def extract_code_with_language(string):
    pattern = r"```([\w\-#]+)?\n([\s\S]*?)\n?```"
    matches = re.findall(pattern, string)
    code_with_language = []

    for match in matches:
        language_name = match[0].strip() if match[0] else None
        code = match[1]
        code_with_language.append({'language': language_name, 'code': code})

    return code_with_language

# use fronted techniques to show the code snippets in a gorgeous way...